# Bias of seasonal forecast

## Import packages

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import regionmask
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download

plt.style.use("seaborn-v0_8-notebook")

## Define Parameters

In [ ]:
# Time
year_forecast = 2023
year_start_hindcast = 1993
year_stop_hindcast = 2016

# Variable
variables = [
    "2m_temperature",
    "total_precipitation",
    "surface_solar_radiation_downwards",
    "10m_wind_speed",
    "2m_dewpoint_temperature",
]

# Centres
centres = {
    "cmcc": {"system": "35"},
    "dwd": {"system": "21"},
    "eccc": {"system": "3"},
    "ecmwf": {"system": "51"},
    "jma": {"system": "3"},
    "meteo_france": {"system": "8"},
    "ncep": {"system": "2"},
    "ukmo": {"system": "601"},
}
assert all("system" in v for v in centres.values())
assert set(centres) <= {
    "cmcc",
    "dwd",
    "eccc",
    "ecmwf",
    "jma",
    "meteo_france",
    "ncep",
    "ukmo",
}

# Regions
regions = ["WNA", "ENA", "NEB", "WSA", "MED", "EAF", "SAS", "SEA"]
assert set(regions) <= set(regionmask.defined_regions.srex.abbrevs)

## Plot selected regions

In [ ]:
ax = regionmask.defined_regions.srex[regions].plot(
    projection=ccrs.Robinson(),
    label="abbrev",
    add_ocean=True,
)
ax.set_global()

## Define requests

In [ ]:
chunks = {"year": 1, "variable": 1}
common_request = {
    "format": "grib",
    "area": [89.5, -179.5, -89.5, 179.5],
    "year": [str(year) for year in range(year_start_hindcast, year_stop_hindcast + 1)],
}

collection_id_reanalysis = "reanalysis-era5-single-levels-monthly-means"
request_reanalysis = common_request | {
    "product_type": "monthly_averaged_reanalysis",
    "month": [f"{month:02d}" for month in range(1, 13)],
    "time": "00:00",
}

collection_id_seasonal = "seasonal-monthly-single-levels"
request_seasonal = common_request | {
    "product_type": "monthly_mean",
    "leadtime_month": list(map(str, range(1, 7))),
    "month": [f"{month:02d}" for month in range(1, 13)],
    "grid": "1/1",
}

## Functions to cache

In [ ]:
def regionalised_spatial_weighted_mean(ds, region, weights, mean_dims=None):
    mask = regionmask.defined_regions.srex.mask(ds)
    index = regionmask.defined_regions.srex.map_keys(region)
    ds = ds.where((mask == index).compute(), drop=True)
    ds = diagnostics.spatial_weighted_mean(ds, weights=weights)
    if mean_dims:
        return ds.mean(mean_dims, keep_attrs=True)
    return ds

## Download and transform ERA5

In [ ]:
def preprocess(ds):
    return ds.expand_dims("leadtime")


# Get the reanalysis data
datasets = []
for region in regions:
    dataarrays = []
    for variable in variables:
        print(f"{region=} {variable=}")
        ds = download.download_and_transform(
            collection_id_reanalysis,
            request_reanalysis | {"variable": variable},
            chunks=chunks,
            transform_func=regionalised_spatial_weighted_mean,
            transform_func_kwargs={"region": region, "weights": False},
            combine="nested",
        )
        (varname,) = set(ds.data_vars)
        dataarrays.append(ds[varname].rename(variable).drop_vars("leadtime"))
    ds = xr.merge(dataarrays)
    datasets.append(ds.expand_dims(region=[region]).compute())
ds_reanalysis = xr.concat(datasets, "region")

## Download and transform seasonal forecast

In [ ]:
missing_variables = {"eccc": ["2m_dewpoint_temperature"]}

# Get the reanalysis data
datasets = []
for centre, request_kwargs in centres.items():
    for region in regions:
        dataarrays = []
        for variable in variables:
            print(f"{centre=} {region=} {variable=}")
            if variable in missing_variables.get(centre, []):
                print("SKIP")
                continue

            ds = download.download_and_transform(
                collection_id_seasonal,
                request_seasonal
                | {"originating_centre": centre, "variable": variable}
                | request_kwargs,
                chunks=chunks,
                transform_func=regionalised_spatial_weighted_mean,
                transform_func_kwargs={
                    "region": region,
                    "weights": False,
                    "mean_dims": ("realization",),
                },
                backend_kwargs={
                    "time_dims": (
                        "verifying_time",
                        (
                            "indexing_time"
                            if centre in ["ukmo", "jma", "ncep"]
                            else "time"
                        ),
                    )
                },
            )
            if "time" in ds.dims:
                ds = ds.rename(time="verifying_time")
            (varname,) = set(ds.data_vars)
            dataarrays.append(ds[varname].rename(variable))
        ds = xr.merge(dataarrays)
        datasets.append(ds.expand_dims(centre=[centre], region=[region]).compute())
ds_seasonal = xr.merge(datasets)

## Convert units

In [ ]:
m_to_mm = 1.0e3
day_to_s = 60 * 60 * 24

for ds in (ds_seasonal, ds_reanalysis):
    for varname, da in ds.data_vars.items():
        if varname == "total_precipitation" and da.attrs["units"] != "mm/month":
            if da.attrs["units"] == "m":
                factor = m_to_mm
            elif da.attrs["units"] == "m s**-1":
                factor = m_to_mm * day_to_s
            else:
                raise ValueError(f"{da.attrs['units']=}")

            with xr.set_options(keep_attrs=True):
                da *= factor * da["forecast_reference_time"].dt.days_in_month
            da.attrs["units"] = "mm/month"

## Monthly data

In [ ]:
ds_reanalysis = ds_reanalysis.groupby("forecast_reference_time.month").mean(
    keep_attrs=True
)
ds_reanalysis = ds_reanalysis.rename(month="valid_month")

for dim_in, dim_out in zip(["forecast_reference", "verifying"], ["starting", "valid"]):
    ds_seasonal = (
        ds_seasonal.groupby(f"{dim_in}_time.month")
        .mean(keep_attrs=True)
        .rename(month=f"{dim_out}_month")
    )

## Compute bias

In [ ]:
with xr.set_options(keep_attrs=True):
    bias = ds_seasonal - ds_reanalysis
for da in bias.data_vars.values():
    da.attrs["long_name"] = "Bias of " + da.attrs["long_name"]
# Avoid ticks interpolation
bias = bias.assign_coords(
    {name: coord.astype(str) for name, coord in bias.coords.items()}
)

## Plot all biases together

In [ ]:
plot_kwargs = {
    "2m_temperature": {"cmap": "RdBu_r"},
    "total_precipitation": {"cmap": "BrBG"},
}

for varname, da in bias.data_vars.items():
    facet = da.plot(
        col="centre", row="region", x="valid_month", **plot_kwargs.get(varname, {})
    )
    facet.fig.suptitle(f"Seasonal forecast: {da.attrs['long_name']}", y=1.01)
    plt.show()